In [1]:
import cv2
import numpy as np
print("Importing tf")
import tensorflow as tf

Importing tf


In [2]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [3]:
new_model = tf.keras.models.load_model('new_model.h5')
# face_clsfr = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [4]:
import os
prototxtPath=os.path.sep.join([r'C:\Users\Arjun\OneDrive\Desktop\ML\opencv','deploy.prototxt'])
weightsPath=os.path.sep.join([r'C:\Users\Arjun\OneDrive\Desktop\ML\opencv','res10_300x300_ssd_iter_140000.caffemodel'])

In [5]:
faceNet=cv2.dnn.readNet(prototxtPath,weightsPath)

In [6]:
cap = cv2.VideoCapture(0)

In [7]:
cap.set(3, 1208)
cap.set(4, 720)

True

In [8]:
def detect_and_predict_mask(frame,faceNet,maskNet):
    #grab the dimensions of the frame and then construct a blob
    (h,w)=frame.shape[:2]
    blob=cv2.dnn.blobFromImage(frame,1.0,(300,300),(104.0,177.0,123.0))
    
    faceNet.setInput(blob)
    detections=faceNet.forward()
    
    #initialize our list of faces, their corresponding locations and list of predictions
    
    faces=[]
    locs=[]
    preds=[]
    
    
    for i in range(0,detections.shape[2]):
        confidence=detections[0,0,i,2]
    
    
        if confidence>0.5:
        #we need the X,Y coordinates
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY)=box.astype('int')
        
            #ensure the bounding boxes fall within the dimensions of the frame
            (startX,startY)=(max(0,startX),max(0,startY))
            (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
            #extract the face ROI, convert it from BGR to GRAY channel, resize it to 120, 120 and preprocess it
            face=frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
            resized=cv2.resize(face,(120,120))
            normalized=resized/255.0
            reshaped=np.reshape(normalized,(1,120,120,1))
#             preds=new_model.predict(reshaped)
        
            faces.append(reshaped)
            locs.append((startX,startY,endX,endY))
        
        #only make a predictions if atleast one face was detected
        if len(faces)>0:
#             faces=np.array(faces,dtype='float32')
            preds=maskNet.predict(faces,batch_size=12)
        
        return (locs,preds)

In [9]:
source=cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
cap.set(10, 200)

labels_dict={1:'MASK',0:'NO MASK'}
color_dict={1:(0,255,0),0:(0,0,255)}
while(True):

    ret,frame=source.read()
    frame = cv2.flip(frame,1)
    (locs,preds)=detect_and_predict_mask(frame,faceNet,new_model) 

    for (box,pred) in zip(locs,preds):
        (startX,startY,endX,endY)=box
        (withoutMask,mask)=pred
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #display the label and bounding boxes
        cv2.putText(frame,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        
        cv2.rectangle(frame,(startX,startY),(endX,endY),color,2)
        
       
    cv2.imshow('LIVE',frame)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

In [10]:
image = cv2.imread("Photos/Mask.jpeg", 1)

In [11]:
(h, w) = image.shape[:2]

In [12]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [13]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

In [14]:
net.setInput(blob)
detections=net.forward()

In [15]:
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(120,120))
        cv2.imshow("OutPut",face)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
#         face=np.expand_dims(face,axis=0)
        
#         (mask,withoutMask)=model.predict(face)[0]
        
#         #determine the class label and color we will use to draw the bounding box and text
#         label='Mask' if mask>withoutMask else 'No Mask'
#         color=(0,255,0) if label=='Mask' else (0,0,255)
        
#         #include the probability in the label
#         label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
#         #display the label and bounding boxes
#         cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
#         cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
